# Projeto 1

João Antônio Misson Milhorim - 11834331

Reynaldo Coronatto Neto - 12547594

In [119]:
#%pip install glfw
#%pip install PyOpenGL
#%pip install numpy
#%pip install glm
#%pip install Pillow

In [120]:
import os
import math
import glfw
import random
import numpy as np
import OpenGL.GL.shaders

from PIL import Image
from OpenGL.GL import *
from numpy import random

In [121]:
class GLWrapper:
    ''' Classe que faz o setup do OpenGL e permite a visualização de imagens '''
        
    def __init__(self, height=700, width=700, title='', objects=None):
        ''' Construtor responsável por chamar funções que configuram o OpenGL '''
        self.objects = objects
        
        # executando funções
        self.set_keys()
        self.set_increments()
        self.window = self.init_window(height, width, title)
        self.add_events(self.window)
        self.program, self.vertex, self.fragment = self.set_shaders()
        self.compile_shaders(self.vertex, 'vertex')
        self.compile_shaders(self.fragment, 'fragment')
        self.attach_shaders(self.program, self.vertex, self.fragment)
        self.link_program(self.program)
        self.set_textures()
    
    def set_objects(self, objs):
        ''' setter do atributo objects'''
        self.objects = objs
    
    def set_keys(self):
        self.KEY_ARROW_DOWN = 264
        self.KEY_ARROW_UP = 265
        self.KEY_ARROW_LEFT = 263
        self.KEY_ARROW_RIGHT = 262
        self.KEY_A = 65
        self.KEY_S = 83
        self.KEY_D = 68
        self.KEY_W = 87
        self.KEY_X = 88
        self.KEY_Z = 90
        self.KEY_V = 86
        self.KEY_P = 80
        
        self.KEY_1 = 49
        self.KEY_2 = 50
        self.KEY_3 = 51
        self.KEY_4 = 52
        self.KEY_5 = 53
    
    def set_increments(self):
        self.rx_inc = 0.0
        self.ry_inc = 0.0
        self.s_inc = 0.0
        self.tx_inc = 0.0
        self.ty_inc = 0.0
        self.polygonal = False
        self.magnification = False
        
    def get_polygonal(self):
        ''' getter do atributo polygonal '''
        return self.polygonal
    
    def show_object(self, obj_id):
        ''' prepara o cenário para que um objeto seja desenhado, resetando seus atributos e escondendo os outros objetos '''
        self.set_increments()
        objects[obj_id].reset()
        for obj in self.objects:
            if obj == obj_id:
                self.objects[obj].set_visibility(True)
            else:
                self.objects[obj].set_visibility(False)
    
    def init_window(self, height, width, title):
        ''' Inicializa janela '''
        glfw.init()
        glfw.window_hint(glfw.VISIBLE, glfw.FALSE);
        window = glfw.create_window(height, width, title, None, None)
        glfw.make_context_current(window)
        
        return window
        
    def key_event(self, window, key, scancode, action, mods):        
        ''' Callback para cliques no teclado '''
        global s_inc, tx_inc, ty_inc, rx_inc, ry_inc, polygonal, objects

        if action == 0:
            return
        
        if key == self.KEY_W: # sobe objeto no eixo y
            self.ty_inc += 0.0005
        
        if key == self.KEY_S: # desce objeto no eixo y
            self.ty_inc -= 0.0005
        
        if key == self.KEY_A: # objeto para a esquerda
            self.tx_inc -= 0.0005
        
        if key == self.KEY_D: # objeto para direita
            self.tx_inc += 0.0005
        
        if key == self.KEY_ARROW_RIGHT: # rotaciona objeto em torno do eixo y
            self.ry_inc -= 0.001
        
        if key == self.KEY_ARROW_LEFT: # rotaciona objeto em torno do eixo y
            self.ry_inc += 0.001
        
        if key == self.KEY_ARROW_UP: # rotaciona objeto em torno do eixo x
            self.rx_inc -= 0.001
        
        if key == self.KEY_ARROW_DOWN: # rotaciona objeto em torno do eixo x
            self.rx_inc += 0.001
        
        if key == self.KEY_Z: # diminui escala do objeto
            self.s_inc -= 0.00001
        
        if key == self.KEY_X: # aumenta escala do objeto
            self.s_inc += 0.00001
        
        if key == self.KEY_P and action == 1: # ativar ou desativar modo poligonal
            self.polygonal = not self.polygonal
        
        if key == self.KEY_V and action == 1: # ativar ou desativar modo poligonal
            self.maginification = not self.magnification
            
        if key == self.KEY_1 and action == 1: # mostrar o objeto 1 ao apertar o número 1
            self.show_object(1 - 1)

        if key == self.KEY_2 and action == 1: # mostrar o objeto 2 ao apertar o número 2
            self.show_object(2 - 1)
            
        if key == self.KEY_3 and action == 1: # mostrar o objeto 3 ao apertar o número 3
            self.show_object(3 - 1)
            
        if key == self.KEY_4 and action == 1: # mostrar o objeto 4 ao apertar o número 4
            self.show_object(4 - 1)

        if key == self.KEY_5 and action == 1: # mostrar o objeto 5 ao apertar o número 5
            self.show_object(5 - 1)
            
    def add_events(self, window):
        ''' Adiciona callbacks dos eventos '''
        glfw.set_key_callback(window, self.key_event)
    
    def set_shaders(self):
        ''' Adiciona shaders '''
        vertex_code = """
                attribute vec3 position;
                attribute vec2 texture_coord;
                varying vec2 out_texture;

                uniform mat4 mat_transform;        

                void main(){
                    gl_Position = mat_transform * vec4(position,1.0);
                    out_texture = vec2(texture_coord);
                }
                """

        fragment_code = """
                uniform vec4 color;
                varying vec2 out_texture;
                uniform sampler2D samplerTexture;

                void main(){
                    vec4 texture = texture2D(samplerTexture, out_texture);
                    gl_FragColor = texture;
                }
        """

        program  = glCreateProgram()
        vertex   = glCreateShader(GL_VERTEX_SHADER)
        fragment = glCreateShader(GL_FRAGMENT_SHADER)
        
        glShaderSource(vertex, vertex_code)
        glShaderSource(fragment, fragment_code)
        
        return program, vertex, fragment

    def compile_shaders(self, shader, name):
        ''' Compila shaders '''
        glCompileShader(shader)
        if not glGetShaderiv(shader, GL_COMPILE_STATUS):
            error = glGetShaderInfoLog(shader).decode()
            print(error)
            raise RuntimeError(f"Erro de compilacao do {name} Shader")
            
    def attach_shaders(self, program, vertex, fragment):
        ''' Associa shaders '''
        glAttachShader(program, vertex)
        glAttachShader(program, fragment)
        
    def link_program(self, program):
        ''' Faz o link do programa '''
        glLinkProgram(program)
        if not glGetProgramiv(program, GL_LINK_STATUS):
            print(glGetProgramInfoLog(program))
            raise RuntimeError('Linking error')

        glUseProgram(program)
        
    def apply_transformations(self, obj_id):
        ''' aplica transformações em um objeto '''
        self.objects[obj_id].update_mat_transform(self.rx_inc, self.ry_inc,self.s_inc, self.tx_inc, self.ty_inc)
        
        mat = self.objects[obj_id].get_mat_transform()
        
        loc = glGetUniformLocation(self.program, "mat_transform")
        glUniformMatrix4fv(loc, 1, GL_TRUE, mat)
    
    def insert_figure_gpu(self, vertices_list, textures_coord_list, mat_transform=None):
        #print(textures_coord_list)
        ''' Gera um buffer e insere dados vertices e texturas na janela, utiliza matriz
            de transformação, caso seja informada '''
        buffer = glGenBuffers(2)
        
        vertices = np.zeros(len(vertices_list), [("position", np.float32, 3)])
        vertices['position'] = vertices_list
        
        # vertices
        glBindBuffer(GL_ARRAY_BUFFER, buffer[0])
        glBufferData(GL_ARRAY_BUFFER, vertices.nbytes, vertices, GL_DYNAMIC_DRAW)

        stride = vertices.strides[0]
        offset = ctypes.c_void_p(0)

        loc = glGetAttribLocation(self.program, "position")
        glEnableVertexAttribArray(loc)

        glVertexAttribPointer(loc, 3, GL_FLOAT, False, stride, offset)
        
                     
        # texturas
        textures = np.zeros(len(textures_coord_list), [("position", np.float32, 2)]) # duas coordenadas
        textures['position'] = textures_coord_list
        
        glBindBuffer(GL_ARRAY_BUFFER, buffer[1])
        glBufferData(GL_ARRAY_BUFFER, textures.nbytes, textures, GL_STATIC_DRAW)
        
        stride = textures.strides[0]
        offset = ctypes.c_void_p(0)
        
        loc_texture_coord = glGetAttribLocation(self.program, "texture_coord")
        glEnableVertexAttribArray(loc_texture_coord)
        
        glVertexAttribPointer(loc_texture_coord, 2, GL_FLOAT, False, stride, offset)
        
    def set_textures(self, num_textures=10):
        glEnable(GL_TEXTURE_2D)
        textures = glGenTextures(num_textures)

In [122]:
class Object:
    ''' Classe para lidar com objetos 3D'''
    
    def __init__(self, id=0, name='NULL', visibility=False):
        self.id = id
        self.name = name
        self.visibility = visibility
        self.obj_file_name = None
        self.texture_file_name = None
        self.init_vertex = 0
        self.end_vertex = 0
        
        self.dx = 0.0 
        self.dy = 0.0
        
        self.tx = 0.0
        self.ty = 0.0
        
        self.scale = 0.01
        
        self.mat_transform = np.array([ 1.0, 0.0, 0.0, 0.0, 
                                        0.0, 1.0, 0.0, 0.0, 
                                        0.0, 0.0, 1.0, 0.0, 
                                        0.0, 0.0, 0.0, 1.0], np.float32)
    
    def reset(self):
        ''' reseta os atributos do objeto '''
        self.dx = 0.0 
        self.dy = 0.0
        
        self.tx = 0.0
        self.ty = 0.0
        
        self.scale = 0.01
        
        self.mat_transform = np.array([ 1.0, 0.0, 0.0, 0.0, 
                                        0.0, 1.0, 0.0, 0.0, 
                                        0.0, 0.0, 1.0, 0.0, 
                                        0.0, 0.0, 0.0, 1.0], np.float32)
    
    def is_visible(self):
        ''' getter do atributo visibility '''
        return self.visibility
    
    def set_visibility(self, visibility):
        ''' setter do atributo visibility '''
        self.visibility = visibility
    
    def set_obj_file_name(self, obj_file_name):
        ''' setter do atributo obj_file_name '''
        self.obj_file_name = obj_file_name 
    
    def set_texture_file_name(self, texture_file_name):
        ''' setter do atributo texture_file_name '''
        self.texture_file_name = texture_file_name      
        
    def print_object(self):
        ''' imprimi atributos do objeto '''
        print(f'id: {self.id}')
        print(f'name: {self.name}')
        print(f'visibility: {self.visibility}')
        print(f'obj file name: {self.obj_file_name}')
        print(f'texture file name: {self.texture_file_name}')
        
    def update_mat_transform(self, rx_inc, ry_inc, s_inc, tx_inc, ty_inc):
        self.dx += rx_inc
        cos_dx = math.cos(self.dx)
        sin_dx = math.sin(self.dx)
        
        rot_x = np.array([  1.0, 0.0, 0.0, 0.0, 
                            0.0, cos_dx, -sin_dx, 0.0, 
                            0.0, sin_dx, cos_dx, 0.0, 
                            0.0, 0.0, 0.0, 1.0], np.float32)
        
        self.dy += ry_inc
        cos_dy = math.cos(self.dy)
        sin_dy = math.sin(self.dy)
        
        rot_y = np.array([ cos_dy, 0.0, sin_dy, 0.0, 
                         0.0, 1.0, 0.0, 0.0, 
                         -sin_dy, 0.0, cos_dy, 0.0, 
                         0.0, 0.0, 0.0, 1.0], np.float32)
        
        self.scale += s_inc
        scale = np.array([  self.scale, 0.0, 0.0, 0.0, 
                            0.0, self.scale, 0.0, 0.0, 
                            0.0, 0.0, self.scale, 0.0, 
                            0.0, 0.0, 0.0, 1.0], np.float32)
        
        self.tx += tx_inc
        self.ty += ty_inc
        
        if self.tx > 1:
            self.tx %= -1
        elif self.tx < -1:
            self.tx %= 1
            
        if self.ty > 1:
            self.ty %= -1
        elif self.ty < -1:
            self.ty %= 1
        
        translation = np.array([1.0, 0.0, 0.0, self.tx, 
                                0.0, 1.0, 0.0, self.ty, 
                                0.0, 0.0, 1.0, 0.0, 
                                0.0, 0.0, 0.0, 1.0], np.float32)
        
        self.mat_transform = matrix_multiplication(rot_y, rot_x)
        self.mat_transform = matrix_multiplication(scale, self.mat_transform)
        self.mat_transform = matrix_multiplication(translation, self.mat_transform)
    
    
    def get_mat_transform(self):
        return self.mat_transform
        
    def load_object(self):
        model = load_model_from_file(self.obj_file_name)

        self.init_vertex = len(vertices_list) 
        for face in model['faces']:
            for vertice_id in face[0]:
                vertices_list.append(model['vertices'][vertice_id-1] )
            for texture_id in face[1]:
                textures_coord_list.append(model['texture'][texture_id-1] )
        self.end_vertex = len(vertices_list)

        load_texture_from_file(self.id, self.texture_file_name)
        
    def draw_obj(self):
        #define id da textura do modelo
        glBindTexture(GL_TEXTURE_2D, self.id)

        # desenha o modelo
        glDrawArrays(GL_QUADS , self.init_vertex, self.end_vertex - self.init_vertex) ## renderizando

In [123]:
def matrix_multiplication(a,b):
    m_a = a.reshape(4,4)
    m_b = b.reshape(4,4)
    m_c = np.dot(m_a,m_b)
    c = m_c.reshape(1,16)
    return c
    

def load_model_from_file(filename):
    ''' Loads a Wavefront OBJ file. '''
    objects = {}
    vertices = []
    texture_coords = []
    faces = []
    max_values = []

    material = None

    # abre o arquivo obj para leitura
    for line in open(filename, "r"): ## para cada linha do arquivo .obj
        if line.startswith('#'): continue ## ignora comentarios
        values = line.split() # quebra a linha por espaço
        if not values: continue


        ### recuperando vertices
        if values[0] == 'v':
            vertices.append(values[1:4])


        ### recuperando coordenadas de textura
        elif values[0] == 'vt':
            texture_coords.append(values[1:3])

        ### recuperando faces 
        elif values[0] in ('usemtl', 'usemat'):
            material = values[1]
        elif values[0] == 'f':
            face = []
            face_texture = []
            for v in values[1:]:
                w = v.split('/')
                face.append(int(w[0]))
                if len(w) >= 2 and len(w[1]) > 0:
                    face_texture.append(int(w[1]))
                else:
                    face_texture.append(0)

            faces.append((face, face_texture, material))

    model = {}
    model['vertices'] = vertices
    model['texture'] = texture_coords
    model['faces'] = faces

    return model

def load_texture_from_file(texture_id, img_textura):
    glBindTexture(GL_TEXTURE_2D, texture_id)
    glTexParameteri(GL_TEXTURE_2D, GL_TEXTURE_WRAP_S, GL_REPEAT)
    glTexParameteri(GL_TEXTURE_2D, GL_TEXTURE_WRAP_T, GL_REPEAT)
    glTexParameteri(GL_TEXTURE_2D, GL_TEXTURE_MIN_FILTER, GL_LINEAR)
    glTexParameteri(GL_TEXTURE_2D, GL_TEXTURE_MAG_FILTER, GL_LINEAR)
    img = Image.open(img_textura)
    img_width = img.size[0]
    img_height = img.size[1]
    image_data = img.tobytes("raw", "RGB", 0, -1)
    #image_data = np.array(list(img.getdata()), np.uint8)
    glTexImage2D(GL_TEXTURE_2D, 0, GL_RGB, img_width, img_height, 0, GL_RGB, GL_UNSIGNED_BYTE, image_data)


def read_objects_from_dir():
    ''' função para ler os arquivos do diretório e criar um dicionário com os objetos'''
    # Define o caminho do diretório 'obj'
    obj_folder_path = os.path.join(os.path.dirname('./'), 'obj')

    # Inicializa o dicionário de objetos: {'id' : 'classe'}
    objects = {}

    # Iterando sobre cada diretório contido em 'obj'
    for folder_name in os.listdir(obj_folder_path):
        folder_path = os.path.join(obj_folder_path, folder_name)

        # Verifica se o arquivo é um diretório
        if os.path.isdir(folder_path):
            # Inicializa o Objeto
            aux = Object(id=len(objects), name=folder_name, visibility=False)

            # Busca por arquivos '.obj' e '.png' no diretório atual
            for file_name in os.listdir(folder_path):
                file_path = os.path.join(folder_path, file_name)
                if file_name.endswith('.obj'):
                    aux.set_obj_file_name(file_path)
                elif file_name.endswith('.jpg'):
                    aux.set_texture_file_name(file_path) 

            # Adiciona o objeto criado ao dicionario objects
            objects[len(objects)] = aux
            
    return objects

In [124]:
g = GLWrapper(title='Projeto 1')

vertices_list = []
textures_coord_list = []

# Lendo arquivos .obj e .jgp do diretório atual
objects = read_objects_from_dir()

# Imprimindo o resultado da leitura do diretório
for obj in objects:
    objects[obj].print_object()
    print()
    
# Carregando objetos
for obj_id in objects:
    objects[obj_id].load_object()

g.insert_figure_gpu(vertices_list, textures_coord_list)
g.set_objects(objects)

id: 0
name: cap
visibility: False
obj file name: .\obj\cap\cap.obj
texture file name: .\obj\cap\cap.jpg

id: 1
name: cat
visibility: False
obj file name: .\obj\cat\cat.obj
texture file name: .\obj\cat\cat.jpg

id: 2
name: dog
visibility: False
obj file name: .\obj\dog\dog.obj
texture file name: .\obj\dog\dog.jpg

id: 3
name: skull
visibility: False
obj file name: .\obj\skull\skull.obj
texture file name: .\obj\skull\skull.jpg

id: 4
name: toilet
visibility: False
obj file name: .\obj\toilet\toilet.obj
texture file name: .\obj\toilet\toilet.jpg



In [125]:
glfw.show_window(g.window)

glEnable(GL_DEPTH_TEST) ### importante para 3D

while not glfw.window_should_close(g.window):
    glfw.poll_events() 

    glClear(GL_COLOR_BUFFER_BIT | GL_DEPTH_BUFFER_BIT)
    
    glClearColor(1.0, 1.0, 1.0, 1.0)

    for obj_id in g.objects: # para cada objeto
        if g.objects[obj_id].is_visible(): # caso o objeto esteja visível
            
            # verifica se o objeto deve ser mostrado sem textura
            if g.get_polygonal() == True:
                glPolygonMode(GL_FRONT_AND_BACK,GL_LINE)
    
            if g.get_polygonal() == False:
                glPolygonMode(GL_FRONT_AND_BACK,GL_FILL)
            
            # aplica transformações e mostra o objetio
            g.apply_transformations(obj_id)
            g.objects[obj_id].draw_obj()
    
    glfw.swap_buffers(g.window)

glfw.terminate()